In [1]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

In [2]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,untar=True) # untar = true so it untar to florder ~/.keras/datasets/aclImdb

In [3]:
dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
train_dir = os.path.join(os.path.join(dataset_dir),'train')
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [4]:
batch_size = 1024
seed = 123
train_ds = tf.keras.utils.text_dataset_from_directory(train_dir,batch_size=batch_size,validation_split=0.2,subset='training',seed=seed)
val_ds = tf.keras.utils.text_dataset_from_directory(train_dir,batch_size=batch_size,validation_split=0.2,subset='validation',seed=seed)
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [5]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')

In [6]:
vocab_size = 10000
seque_len = 100
embedding_dim = 16
text_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size,standardize='lower_and_strip_punctuation',output_mode='int',output_sequence_length=seque_len)
text_ds = train_ds.map(lambda x,y: x)
text_layer.adapt(text_ds)
model = tf.keras.Sequential([
    text_layer,
    tf.keras.layers.Embedding(vocab_size, embedding_dim, name="embedding"),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(2)
])

In [7]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(train_ds,validation_data=val_ds,epochs=20)

Epoch 1/20
20/20 [==============================] - 3s 142ms/step - loss: 0.6920 - accuracy: 0.5958 - val_loss: 0.6898 - val_accuracy: 0.6692
Epoch 2/20
20/20 [==============================] - 2s 85ms/step - loss: 0.6869 - accuracy: 0.6740 - val_loss: 0.6831 - val_accuracy: 0.7012
Epoch 3/20
20/20 [==============================] - 2s 85ms/step - loss: 0.6782 - accuracy: 0.7003 - val_loss: 0.6721 - val_accuracy: 0.7106
Epoch 4/20
20/20 [==============================] - 2s 82ms/step - loss: 0.6642 - accuracy: 0.7182 - val_loss: 0.6558 - val_accuracy: 0.7208
Epoch 5/20
20/20 [==============================] - 2s 82ms/step - loss: 0.6439 - accuracy: 0.7387 - val_loss: 0.6336 - val_accuracy: 0.7356
Epoch 6/20
20/20 [==============================] - 2s 80ms/step - loss: 0.6159 - accuracy: 0.7611 - val_loss: 0.6043 - val_accuracy: 0.7516
Epoch 7/20
20/20 [==============================] - 2s 81ms/step - loss: 0.5804 - accuracy: 0.7818 - val_loss: 0.5707 - val_accuracy: 0.7708
Epoch 8/20
2

In [17]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = text_layer.get_vocabulary()
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')
for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()